<h1 style="font-size: 2.75em; color: indianred; background-color: khaki; padding: 10px; border-radius: 5px;">
  <b>Unmanned aerial vehicle (UAV) data processing pipeline for periodical data collections at the experimental field station of the National Chung Hsing University (NCHU), Taiwan - 
    <span style="background-color: yellow; color: indianred; font-weight: bold;">2024</span>
  </b> 
</h1>

<h1 style="font-size: 1.75em; color: indianred;">
  <b>1. Clip Digital Elevation Models (DEMs) computed from Agisoft outputs folder</b>
</h1>

In [ ]:

import os
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from tqdm import tqdm

# Directories
input_dir = r"path to your DEMs"
shapefile_path = r"path to your shapefile delimiting the field area"
output_dir = r"path to save your clipped DEMs"

# Create destination directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load shapefile
rois = gpd.read_file(shapefile_path)

# User input for file prefixes
file_prefixes = input("Please enter the first four characters of the DEM files you want to process, separated by commas, or type 'all' to process all DEMs: ")
file_prefix_list = [prefix.strip() for prefix in file_prefixes.split(',')]

def clip_dems(input_directory, output_directory):
# Collect files matching the prefix condition
    all_files = [
        os.path.join(root, file)
        for root, dirs, files in os.walk(input_directory)
        for file in files if file.endswith('.tif')
        and (file_prefixes.lower() == 'all' or any(file.startswith(prefix) for prefix in file_prefix_list))
    ]
    total_iterations = len(all_files) * len(rois)

# Clipping loop with progress bar
    with tqdm(total=total_iterations, desc="Clipping DEMs", unit="task") as pbar:
        for image_path in all_files:
            with rasterio.open(image_path) as src:
                for idx, row in rois.iterrows():
                    try:
# Clip raster by ROI
                        out_image, out_transform = mask(src, [row['geometry']], crop=True)
                        out_meta = src.meta.copy()
                        out_meta.update({
                            "driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform,
                            "count": out_image.shape[0]
                        })

# Save with same name as original DEM
                        output_file_name = os.path.basename(image_path)
                        out_image_path = os.path.join(output_directory, output_file_name)

                        with rasterio.open(out_image_path, "w", **out_meta) as dest:
                            dest.write(out_image)

                    except MemoryError:
                        print(f"MemoryError with file: {image_path}. Skipping...")
                        continue
                    finally:
                        pbar.update(1)

# Run clipping
clip_dems(input_dir, output_dir)

<h1 style="font-size: 1.75em; color: indianred;">
  <b>2. Crop Surface Models (CSMs) calculation</b>
</h1>

In [ ]:

import os
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from tqdm import tqdm

dem_raster_path = r'path to your Digital Terrain Model or DTM'
input_dir = r'path to your clipped DTM files'
output_dir = r'path to save your CSM files'

os.makedirs(output_dir, exist_ok=True)

if 'file_prefixes' not in globals() or 'file_prefix_list' not in globals():
    file_prefixes = input("Enter first 4 characters of files to process (comma-separated) or 'all': ")
    file_prefix_list = [prefix.strip() for prefix in file_prefixes.split(',')]

with rasterio.open(dem_raster_path) as dem_src:
    dem_crs = dem_src.crs
    dem_transform = dem_src.transform
    dem_width = dem_src.width
    dem_height = dem_src.height
    dem_meta = dem_src.meta.copy()
    dem_resolution = (dem_height, dem_width)

    input_files = [
        f for f in os.listdir(input_dir)
        if f.endswith('.tif') and (file_prefixes.lower() == 'all' or any(f.startswith(prefix) for prefix in file_prefix_list))
    ]

    for filename in tqdm(input_files, desc="CSM Calculation", unit="file"):
        input_path = os.path.join(input_dir, filename)
        base_name = os.path.splitext(filename)[0]
        output_name = f"{base_name}_csm.tif"
        output_path = os.path.join(output_dir, output_name)

        if os.path.exists(output_path):
            continue

        with rasterio.open(input_path) as src:
            if src.crs != dem_crs:
                raise ValueError(f"CRS mismatch: {input_path}")
            input_data = src.read(1)
            resampled_data = np.empty(dem_resolution, dtype=np.float32)

            reproject(
                source=input_data,
                destination=resampled_data,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dem_transform,
                dst_crs=dem_crs,
                resampling=Resampling.bilinear
            )

            dem_data = dem_src.read(1)
            result_data = resampled_data - dem_data

            out_meta = dem_meta.copy()
            out_meta.update({"dtype": 'float32', "count": 1, "nodata": np.nan, "compress": "lzw"})

            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(result_data, 1)

csm_files = [
    f for f in os.listdir(output_dir)
    if f.endswith('_csm.tif') and (file_prefixes.lower() == 'all' or any(f.startswith(prefix) for prefix in file_prefix_list))
]

for filename in tqdm(csm_files, desc="Masking CSMs", unit="file"):
    input_path = os.path.join(output_dir, filename)
    base_name = filename.replace('_csm.tif', '')
    output_name = f"{base_name}_csm_masked.tif"
    output_path = os.path.join(output_dir, output_name)

    if os.path.exists(output_path):
        continue

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        masked = np.where(data < 0, np.nan, data)

        out_meta = src.meta.copy()
        out_meta.update({"dtype": 'float32', "count": 1, "nodata": np.nan, "compress": "lzw"})

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(masked, 1)

# Clean up filenames: remove any "_dem" from file names
for filename in os.listdir(output_dir):
    if "_dem" in filename:
        old_path = os.path.join(output_dir, filename)
        new_filename = filename.replace("_dem", "")
        new_path = os.path.join(output_dir, new_filename)
        os.rename(old_path, new_path)

# Clip all outputs
for filename in os.listdir(output_dir):
    if filename.endswith('.tif'):
        raster_path = os.path.join(output_dir, filename)
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, rois.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

# Overwrite original file with clipped version
        with rasterio.open(raster_path, "w", **out_meta) as dest:
            dest.write(out_image)

<h1 style="font-size: 1.75em; color: indianred;">
  <b>3. Volumes computation</b> 
</p>

In [ ]:

import os
import rasterio
from tqdm import tqdm

input_dir = r'path to your CSM files'
output_dir = r'path to save the volume raster files'

os.makedirs(output_dir, exist_ok=True)

# Ask for file prefixes
if 'file_prefixes' not in globals() or 'file_prefix_list' not in globals():
    file_prefixes = input("Enter first 4 characters of files to process (comma-separated) or 'all': ")
    file_prefix_list = [prefix.strip() for prefix in file_prefixes.split(',')]

files_to_process = [
    f for f in os.listdir(input_dir)
    if f.endswith('_masked.tif') and (file_prefixes.lower() == 'all' or any(f.startswith(prefix) for prefix in file_prefix_list))
]

for filename in tqdm(files_to_process, desc="Computing Volumes", unit="file"):
    input_path = os.path.join(input_dir, filename)
    base_name = filename.replace('_csm_masked', '').replace('_masked', '').replace('.tif', '')
    output_name = f"{base_name}_vol.tif"
    output_path = os.path.join(output_dir, output_name)

    if os.path.exists(output_path):
        continue

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        pixel_size = src.transform[0]
        pixel_area = pixel_size ** 2
        volume_data = data * pixel_area

        out_meta = src.meta.copy()
        out_meta.update({"dtype": 'float32', "count": 1, "compress": "lzw"})

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(volume_data, 1)

<h1 style="font-size: 1.75em; color: indianred;">
  <b>4. Extraction of height and volume information by plot ("plot_name")</b>

<h1 style="font-size: 1.5em; color: salmon;">
  Extraction of height information as the quantile 95 of the values within the area of interest of the plant
</h1>

In [ ]:

import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import glob
import os
import numpy as np
from tqdm import tqdm

# Define file paths
raster_folder = r"path to your CSM files"
shapefile_path = r"path to your shapefiles containing one polygon per plant"
output_csv_path = r"path to save your height results"

# Load shapefile
shapefile = gpd.read_file(shapefile_path)

# List raster files
raster_files = glob.glob(os.path.join(raster_folder, "*_masked.tif"))

# Initialize data list
data = []

# Loop through raster files
for raster_path in tqdm(raster_files, desc="Processing Raster Files", unit="file"):
    raster_name = os.path.basename(raster_path).replace('.tif', '')
    date_str = '2024' + raster_name[:4]  # Assumes MMDD format  YYYYMMDD

    with rasterio.open(raster_path) as src:
        nodata = src.nodata

        for _, row in shapefile.iterrows():
            geometry = row['geometry']
            plot_name = row['plt_name']
            harvest = row.get('harvest', 'Unknown')
            no_plant = row.get('no_plant', np.nan)  # <- Extract no_plant column

            try:
                raster_date_int = int(raster_name[:4])
            except ValueError:
                raster_date_int = 0

            quantile_95 = np.nan

            if not (harvest == 'IH' and raster_date_int >= 725):
                try:
                    out_image, _ = mask(src, [geometry], crop=True)
                    valid_values = out_image.flatten()
                    valid_values = valid_values[(valid_values != nodata) & (valid_values >= 0)]

                    if valid_values.size > 0:
                        quantile_95 = np.quantile(valid_values, 0.95)
                except Exception:
                    pass  # quantile_95 remains NaN

            data.append([plot_name, no_plant, date_str, quantile_95])

# Convert to DataFrame and save
df = pd.DataFrame(data, columns=['plot_name', 'no_plant', 'date', 'height_quantile_95'])
df.to_csv(output_csv_path, index=False)
print(" Saved:", output_csv_path)

<h1 style="font-size: 1.5em; color: salmon;">
  Extraction of volume information as the sum of the volume of all the pixels within the area of interest of the plant
  <span style="font-size: 1em; line-height: 1; color: grey;">
</h1>

In [ ]:

import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import glob
import os
import numpy as np
from tqdm import tqdm

# Paths
raster_folder = r"path to your volume raster files"
shapefile_path = r"path to your shapefiles containing one polygon per plant"
output_csv = r"path to save your volume results"

# Load shapefile
shapefile = gpd.read_file(shapefile_path)

# List all volume rasters
raster_files = glob.glob(os.path.join(raster_folder, "*_vol.tif"))

# Collector
data = []

# Process rasters
for raster_path in tqdm(raster_files, desc="Processing Volume Rasters", unit="file"):
    raster_name = os.path.basename(raster_path).replace('.tif', '')
    date_str = '2024' + raster_name[:4]

    with rasterio.open(raster_path) as src:
        nodata = src.nodata

        for _, row in shapefile.iterrows():
            geometry = row['geometry']
            plot_name = row['plt_name']
            harvest = row.get('harvest', 'Unknown')
            no_plant = row.get('no_plant', np.nan)

            try:
                raster_date_int = int(raster_name[:4])
            except ValueError:
                raster_date_int = 0

            volume = np.nan
            if not (harvest == 'IH' and raster_date_int >= 725):
                try:
                    out_image, _ = mask(src, [geometry], crop=True)
                    valid_values = out_image.flatten()
                    valid_values = valid_values[(valid_values != nodata) & (valid_values >= 0)]
                    if valid_values.size > 0:
                        volume = np.sum(valid_values)
                except Exception:
                    pass

            data.append([plot_name, no_plant, date_str, volume])

# Save
df = pd.DataFrame(data, columns=['plot_name', 'no_plant', 'date', 'volume'])
df.to_csv(output_csv, index=False)
print(" Saved:", output_csv)